In [293]:
import pandas as pd
from sklearn import preprocessing
import keras
from keras.utils import np_utils
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.layers import Input, Dense, LSTM, Embedding
from keras.models import Model
from keras.optimizers import Adam

# Data Processing

In [294]:
filepath = '../../Datasets/SICK/SICK.txt'
data = pd.read_csv(filepath, sep='\t')

In [295]:
data.head()

,pair_ID,sentence_A,sentence_B,entailment_label,relatedness_score,entailment_AB,entailment_BA,sentence_A_original,sentence_B_original,sentence_A_dataset,sentence_B_dataset,SemEval_set
0,1,A group of kids is playing in a yard and an ol...,A group of boys in a yard is playing and a man...,NEUTRAL,4.5,A_neutral_B,B_neutral_A,"A group of children playing in a yard, a man i...","A group of children playing in a yard, a man i...",FLICKR,FLICKR,TRAIN
1,2,A group of children is playing in the house an...,A group of kids is playing in a yard and an ol...,NEUTRAL,3.2,A_contradicts_B,B_neutral_A,"A group of children playing in a yard, a man i...","A group of children playing in a yard, a man i...",FLICKR,FLICKR,TRAIN
2,3,The young boys are playing outdoors and the ma...,The kids are playing outdoors near a man with ...,ENTAILMENT,4.7,A_entails_B,B_entails_A,"The children are playing outdoors, while a man...","The children are playing outdoors, while a man...",FLICKR,FLICKR,TRAIN
3,4,The young boys are playing outdoors and the ma...,There is no boy playing outdoors and there is ...,CONTRADICTION,3.6,A_contradicts_B,B_contradicts_A,"The children are playing outdoors, while a man...","The children are playing outdoors, while a man...",FLICKR,FLICKR,TRIAL
4,5,The kids are playing outdoors near a man with ...,A group of kids is playing in a yard and an ol...,NEUTRAL,3.4,A_neutral_B,B_neutral_A,"A group of children playing in a yard, a man i...","The children are playing outdoors, while a man...",FLICKR,FLICKR,TRAIN


In [296]:
num_sentences = data.shape[0]
num_labels = 3

We will extract the Premise and Hypothesis along with the Entailment Label to construct our dataset.

In [297]:
X = data[['sentence_A','sentence_B']]
Y = data['entailment_label']

We can process our label encoding now, but our sentences are not in a format which can be encoded.

In [298]:
le = preprocessing.LabelEncoder().fit(Y)
Y = np_utils.to_categorical(le.transform(Y))
print(Y)

[[0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 ...
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]]


sentence_A holds the Premise while sentence_B holds the Hypothesis data.

To tokenize our corpus we will need to stack these two columns into a single series.

In [299]:
stacked = pd.concat([X['sentence_A'], X['sentence_B']], axis=0)

In [300]:
# Convert question corpus into sequential encoding for LSTM
vocab_size = 2048
sequence_length = 32
tokenizer = Tokenizer(num_words=vocab_size)

tokenizer.fit_on_texts(stacked)
sequences = tokenizer.texts_to_sequences(stacked)
x_text = sequence.pad_sequences(sequences, maxlen=sequence_length)
premise_sequences = x_text[:data.shape[0]]
hypothesis_sequences = x_text[data.shape[0]:]

# Neural Network

In [ ]:
samples = 60000
epochs = 100
batch_size = 64
learning_rate = .00025

In [302]:
embedding_layer = Embedding(vocab_size, 128)

premise_inputs = Input(shape=(sequence_length,))
premise_embedding = embedding_layer(premise_inputs)
premise_outputs, state_h, state_c = LSTM(128, return_state=True)(premise_embedding)

hypothesis_inputs = Input(shape=(sequence_length,))
hypothesis_embedding = embedding_layer(hypothesis_inputs)
hypothesis_outputs = LSTM(128)(hypothesis_embedding, initial_state=[state_h, state_c])

outputs = Dense(num_labels, activation='softmax')(hypothesis_outputs)

In [303]:
model = Model(inputs=[premise_inputs, hypothesis_inputs], outputs=outputs)
print(model.summary())

model.compile(optimizer=Adam(lr=learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit([premise_sequences, hypothesis_sequences], Y, batch_size=batch_size, epochs=epochs, shuffle=True)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_23 (InputLayer)           (None, 32)           0                                            
__________________________________________________________________________________________________
input_22 (InputLayer)           (None, 32)           0                                            
__________________________________________________________________________________________________
embedding_30 (Embedding)        (None, 32, 128)      262144      input_22[0][0]                   
                                                                 input_23[0][0]                   
__________________________________________________________________________________________________
lstm_47 (LSTM)                  [(None, 128), (None, 131584      embedding_30[0][0]               
__________

9840/9840 [==============================] - 19s 2ms/step - loss: 0.2019 - acc: 0.9122
Epoch 64/100
9840/9840 [==============================] - 24s 2ms/step - loss: 0.1919 - acc: 0.9165
Epoch 65/100
9840/9840 [==============================] - 20s 2ms/step - loss: 0.1925 - acc: 0.9177
Epoch 66/100
9840/9840 [==============================] - 19s 2ms/step - loss: 0.1939 - acc: 0.9185
Epoch 67/100
9840/9840 [==============================] - 19s 2ms/step - loss: 0.1849 - acc: 0.9218
Epoch 68/100
9840/9840 [==============================] - 19s 2ms/step - loss: 0.1767 - acc: 0.9242
Epoch 69/100
9840/9840 [==============================] - 22s 2ms/step - loss: 0.1787 - acc: 0.9262
Epoch 70/100
9840/9840 [==============================] - 20s 2ms/step - loss: 0.1832 - acc: 0.9248
Epoch 71/100
9840/9840 [==============================] - 19s 2ms/step - loss: 0.1726 - acc: 0.9279
Epoch 72/100
9840/9840 [==============================] - 21s 2ms/step - loss: 0.1660 - acc: 0.9311
Epoch 73/100
